## AI Server SDK for Python

### Install Dependencies

In [ ]:
%pip install --upgrade ai-server-sdk
%pip install python-dotenv

## Server Connection & Insights

### Creating the Server Connection

In [ ]:
import os
from ai_server import ServerClient
from dotenv import load_dotenv

load_dotenv('../.env')
SECRET_KEY = os.getenv('DEV_SECRET_KEY')
ACCESS_KEY = os.getenv('DEV_ACCESS_KEY')

# Here we connect to the demo instance of CFG, but you would connect to your own instance
connection_url = os.getenv('DEV_URL')

# This object creates a connection to the CFG server
server_connection = ServerClient(base = connection_url, access_key = ACCESS_KEY, secret_key = SECRET_KEY)

# Check if you are connected to the server
is_connected = server_connection.connected
print(f"Am I connected to the server? {is_connected}")

# Initilizing the server creates a new insight which is accessible through .cur_insight
my_insight = server_connection.cur_insight
print(f"This is my current insight ID: {my_insight}")

### Create new insights

In [ ]:
# You can create new insights by calling the .make_new_insight() method
new_insight = server_connection.make_new_insight() # This becomes your active insight

print(f"My current insight is now: {server_connection.cur_insight}")

### Listing open insights

In [ ]:
my_open_insights = server_connection.get_open_insights()

print(f"These are my open insights: {my_open_insights}")

### Dropping insights 

In [ ]:
# You can drop all of your insights with the .drop_insights() method and passing in the list of insights
# Here we drop all of our open insight ids 
server_connection.drop_insights(my_open_insights) # This will create a new insight id since there must be an active insight

print(f"Here are my insight IDs after dropping: {server_connection.get_open_insights()}")

### Python trick to check attributes

In [ ]:
attributes = dir(server_connection)
# Find all of the attributes available to the server connection object
attributes = [attr for attr in attributes if not attr.startswith('__')]

print(f"Here are the attributes of the server connection object: {attributes}")

## ModelEngine

### Basic Inferencing

In [ ]:
from ai_server import ModelEngine

# An ID for the specific LLM you want to interact with
llm_id = "4acbe913-df40-4ac0-b28a-daa5ad91b172"

# We need to pass the engine id and an insight id to the ModelEngine object
llm = ModelEngine(engine_id=llm_id, insight_id = server_connection.cur_insight)

question = "What weighs more, a pound of feathers or a pound of bricks?"

# The ask method sends a question to the model and returns the response
try:
    answer = llm.ask(question)
except Exception as e:
    print(f"An error occured: {e}")

print(answer['response'])

print("We also get acess to: ")
print(f"messageId: {answer['messageId']}")
print(f"roomId: {answer['roomId']}")
print(f"numberOfTokensInPrompt", answer['numberOfTokensInPrompt'])
print(f"numberOfTokensInResponse", answer['numberOfTokensInResponse'])


### Advanced Inferencing with parameters

In [ ]:
# We can add parameters to our requests such as context, history, max_new_tokens, repetition_penalty, seed, temperature, top_k, top_p, truncate, typical_p
params = {
    "temperature": 0.9, 
    "max_new_tokens": 200,
    "context": "You are a first grade teacher that uses language and explanations easy for children to understand."
    }

# Pass your parameters as a dictionary to the ask method using the param_dict argument
new_answer = llm.ask(question, param_dict=params)

print(new_answer['response'])

### Using chat history

In [ ]:

# We can pass our own history to the ask method
# You can use as many dictionaries as you want in the history list but each one will add tokens to the request
history = [
    {"role": "user", "content": question},
    {"role": "assistant", "content": new_answer['response']}
]

# Here we change the context to a college professor and pass our chat history
params = {
    "temperature": 0.9, 
    "max_new_tokens": 200,
    "context": "You are college professor who provides complex answers backed by science.",
    "history": history
}

# Our new question references the chat history
new_question = "Can you explain your previous answer in more detail?"

new_answer = llm.ask(new_question, params)

print(new_answer['response'])

## VectorEngine

### Adding documents

In [ ]:
from ai_server import VectorEngine

# Using a FAISS vector engine
vector_engine_id = '1222b449-1bc6-4358-9398-1ed828e4f26a'

# We initialize the VectorEngine object with the engine id and the current insight id
faiss_vector_engine = VectorEngine(engine_id=vector_engine_id, insight_id=server_connection.cur_insight)

file_path = "./ai-whitehouse.pdf"

# We can add documents to the faiss index with the addDocument method
faiss_vector_engine.addDocument(file_paths = [file_path])

### Listing uploaded documents

In [ ]:
# Fetch a list of uploaded documents
my_documents = faiss_vector_engine.listDocuments()

print(my_documents)

### Performing a nearest neighbor search

In [ ]:
query = "How does the document define machine learning?"

# Find the closest match(es) between the question bassed in and the embedded documents using Euclidena Distance.
nearest_neighbor = faiss_vector_engine.nearestNeighbor(search_statement=query, limit = 3, insight_id = server_connection.cur_insight)

for index, result in enumerate(nearest_neighbor):
    print(f"Result {index + 1}")
    print(f"SCORE: {result['Score']}")
    print(f"TOKENS: {result['Tokens']}")
    print(f"CONTENT: {result['Content']}")

### Removing documents

In [ ]:
# Names of the files we want to remove
file_names = ['ai-whitehouse.pdf']

faiss_vector_engine.removeDocument(file_names = file_names)

print(faiss_vector_engine.listDocuments())

## DatabaseEngine

### Querying a database

In [ ]:
from ai_server import DatabaseEngine

# An example H2 diabetes database
db_engine_id = "950eb187-e352-444d-ad6a-6476ed9390af"

# Connect to the database by passing the engine id and the current insight id
db = DatabaseEngine(engine_id=db_engine_id, insight_id=server_connection.cur_insight)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE height < 62 AND weight > 200")

print(query)

### Inserting data

In [ ]:
insert = "INSERT INTO diabetes (height, weight, location) VALUES (65, 200, 'Rosslyn')"

# NOTE you will need to be an author or editor of the database to insert or delete data
db.insertData(query = insert)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE location = 'Rosslyn'")

print(query)

### Deleting data

In [ ]:
remove = "DELETE FROM diabetes WHERE location = 'Rosslyn'"

db.removeData(query = remove)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE location = 'Rosslyn'")

print(query)

## FunctionEngine

In [ ]:
from ai_server import FunctionEngine

# Weather function engine
weather_id = '1563ddbb-2f77-4094-93f4-083431f15cbc'

function = FunctionEngine(engine_id=weather_id, insight_id=server_connection.cur_insight)

# Parameters will change based on the function you are using
output = function.execute({"lat":"37.540","lon":"77.4360"})

print(output)

## StorageEngine

### Listing Files

In [ ]:
from ai_server import StorageEngine

# Example S3 bucket
storage_engine_id = "2d905aa3-b703-4c98-8133-5bcaefddac1e"

storageEngine = StorageEngine(engine_id = storage_engine_id, insight_id = server_connection.cur_insight)

s3_storage_path = "/my-new-test-folder/"

# A list of the files in the given path
my_dir = storageEngine.list(storagePath = s3_storage_path)

for file in my_dir:
    print (f"-- {file}")

# A list of details about the files in the given path
my_dir_details = storageEngine.listDetails(storagePath = '/my-new-test-folder/')

for file_details in my_dir_details:
    print (f"-- {file_details}")


## Running Pixels

In [ ]:
# You can use the active server connection to run pixels

simple_pixel_response = server_connection.run_pixel('1+1')

print('Simple Response --')
print(simple_pixel_response)

full_pixel_response = server_connection.run_pixel('1+1', full_response=True)

print('Full Response --')
print(full_pixel_response)

### LLM Chat with pixels example

In [ ]:

llm_id = "4acbe913-df40-4ac0-b28a-daa5ad91b172"

chat = server_connection.run_pixel(
    f"LLM ( engine = [ '{llm_id}' ] , command = [ '<encode>What is the capital of Connecticut</encode>' ] , paramValues = [ {{ 'max_new_tokens' : 200 , 'temperature' : 0.3 }} ] )"
)
print(chat)